# EX-10 Translator seq2seq

### 곽상혁

### 2022-12-09 (금)

https://github.com/docosa2/

## 모듈 임포트

In [44]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np
import pandas as pd

print(tf.__version__)

2.9.0


실습에서 구현한 번역기는 글자 단위(Character-level)에서 구현된 번역기였습니다. 하지만 실제 번역기의 경우에는 글자 단위가 아니라 단어 단위(Word-level)에서 구현되는 것이 좀 더 보편적입니다.

동일한 데이터셋을 사용하면서 글자 단위와는 다른 전처리와 to_categorical() 함수가 아닌 임베딩 층(Embedding layer)를 추가하여 단어 단위의 번역기를 완성시켜보겠습니다. 하지만, 단어 단위로 할 경우에는 단어의 개수가 글자 단위로 했을 경우와 비교하여 단어장의 크기(Vocabulary) 크기도 커지고, 학습 속도도 좀 더 느려집니다. 학습과 테스트 시의 원활한 진행을 위해서 데이터에서 상위 33,000개의 샘플만 사용해주세요.

33000개 중 3000개는 테스트 데이터로 분리하여 모델을 학습한 후에 번역을 테스트 하는 용도로 사용합니다.

### 데이타 파일 읽어오기

In [45]:
import os
file_path = os.getenv('HOME')+'/aiffel/translator_seq2seq/data/fra.txt'
lines = pd.read_csv(file_path, names=['eng', 'fra', 'cc'], sep='\t')
print('전체 샘플의 수 :',len(lines))
lines.sample(5) #샘플 5개 출력

전체 샘플의 수 : 197463


,eng,fra,cc
65172,I'm glad you came today.,Je suis contente que tu sois venue aujourd'hui.,CC-BY 2.0 (France) Attribution: tatoeba.org #3...
5885,That was Tom.,C'était Tom.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
119794,The weather is beautiful today.,Il fait beau aujourd'hui.,CC-BY 2.0 (France) Attribution: tatoeba.org #6...
116101,He wetted his towel with water.,Il a mouillé sa serviette avec de l'eau.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
63148,He speaks broken French.,Il écorche le français.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...


### 중복 제거

In [46]:
lines = lines.drop_duplicates(subset='eng')
print('중복 제거 후 전체 샘플의 수 :',len(lines))

중복 제거 후 전체 샘플의 수 : 138912


### 불필요한 칼럼 제거

In [47]:
lines = lines[['eng', 'fra']][-33000:].reset_index(drop=True)

In [48]:
lines

,eng,fra
0,I'd like to talk about your situation.,J'aimerais discuter de votre situation.
1,I'd like to talk to the hotel manager.,J'aimerais parler au directeur de l’hôtel.
2,I'd like to thank everyone who helped.,J'aimerais remercier tous ceux qui ont aidé.
3,I'd like you to meet a friend of mine.,J'aimerais que tu rencontres l'un de mes amis.
4,I'd like you to tell me what happened.,J'aimerais que vous me disiez ce qui s'est passé.
...,...,...
32995,A carbon footprint is the amount of carbon dio...,Une empreinte carbone est la somme de pollutio...
32996,Death is something that we're often discourage...,La mort est une chose qu'on nous décourage sou...
32997,Since there are usually multiple websites on a...,Puisqu'il y a de multiples sites web sur chaqu...
32998,If someone who doesn't know your background sa...,Si quelqu'un qui ne connaît pas vos antécédent...


### 셔플

In [49]:
lines = lines.iloc[np.random.permutation(lines.index)].reset_index(drop=True)

In [50]:
lines

,eng,fra
0,I forgot to turn off the TV before going to bed.,J'ai oublié d'éteindre la télé avant d'aller m...
1,Tom is approximately the same weight as Mary.,Tom fait environ le même poids que Mary.
2,"If you can't stand the heat, get out of the ki...","Si vous ne supportez pas la chaleur, sortez de..."
3,Has anyone in your family ever been arrested?,Est-ce que quelqu'un dans votre famille a déjà...
4,I have to finish up some things before I go.,Je dois finir deux trois trucs avant d'y aller.
...,...,...
32995,How can you treat your own mother like that?,Comment peux-tu traiter ta propre mère comme ça ?
32996,Please do not write in this library book.,Veuillez ne pas écrire dans ce livre de la bib...
32997,The doctor advised me to take up some sport to...,Le médecin m'a recommandé de me mettre à faire...
32998,I don't recognize this shirt. Whose is it?,Je ne reconnais pas cette chemise. À qui appar...


### Step 1. 정제, 정규화, 전처리 (영어, 프랑스어 모두!)
---

글자 단위가 아닌 단어 단위의 번역기를 하기 위해서는 글자 단위에서는 신경쓰지 않았던 몇 가지 추가적인 전처리가 필요합니다.

1. 구두점(Punctuation)을 단어와 분리해주세요.

일반적으로 영어권 언어의 경우에는 띄어쓰기 단위로 단어를 분리합니다. 토큰화(Tokenization) 라고도 불리는 이 작업은 어디서부터 어디까지가 하나의 단어인지를 구분하는 작업인데요, 그런데 띄어쓰기를 해주기 전에 구두점을 분리하는 작업이 필요할 때가 있습니다.
예를 들어서 'he is a good boy!'라는 문장이 있을 때, 이를 띄어쓰기 단위로 토큰화한다면 ['he', 'is', 'a', 'good', 'boy!']가 됩니다. 그런데 실제로 !는 boy와 붙어있는 한 단어가 아니므로 좀 더 올바른 전처리는 ['he', 'is', 'a', 'good', 'boy', '!']가 맞습니다.
!나 ? 또는 온점과 같은 특수문자들을 구두점(punctuation)이라고 부릅니다. 이들을 토큰화하기 전에 단어와 미리 분리시켜주세요!

> 분리 전 : he is a Good boy!  
> 분리 후 : he is a Good boy !

In [51]:
import re

def Seperate_punctuations(sentence: str) -> str:
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) 
    return sentence

In [52]:
assert Seperate_punctuations('This Is, So To Speak, A Kind Of A Test Sentence.') == ('This Is ,  So To Speak ,  A Kind Of A Test Sentence . '), 'Error in converter to seperate punctuations.'

In [53]:
def Sepearate_entire_punctuations_for(df: pd.DataFrame) -> None:
    for index, row in df.iterrows():
        row['eng'] = Seperate_punctuations(row['eng'])
        row['fra'] = Seperate_punctuations(row['fra'])

In [54]:
Sepearate_entire_punctuations_for(lines)

In [55]:
lines.head()

,eng,fra
0,I forgot to turn off the TV before going to be...,J'ai oublié d'éteindre la télé avant d'aller m...
1,Tom is approximately the same weight as Mary .,Tom fait environ le même poids que Mary .
2,"If you can't stand the heat , get out of the ...","Si vous ne supportez pas la chaleur , sortez ..."
3,Has anyone in your family ever been arrested ?,Est-ce que quelqu'un dans votre famille a déjà...
4,I have to finish up some things before I go .,Je dois finir deux trois trucs avant d'y aller .


2. 소문자로 바꿔주세요.

기계가 보기에는 스펠링이 같더라도 대문자로 된 단어와 소문자로 된 단어는 서로 다른 단어입니다. 예를 들어 'Good'과 'good'은 기계가 보기에는 다른 단어입니다. 그래서 모든 문장에 대해서 전부 영어로 바꿔주는 작업을 하겠습니다.

> 변환 전 : he is a Good boy !  
> 변환 후 : he is a good boy !

In [56]:
import re

def Convert_to_lower_case(sentence: str) -> str:
    sentence = sentence.lower().strip()
    return sentence

In [57]:
assert Convert_to_lower_case('This Is A Test Sentence.') == ('this is a test sentence.'), 'Error in converter for lower case.'

In [58]:
def Convert_to_lower_cases_for(df: pd.DataFrame) -> None:
    for index, row in df.iterrows():
        row['eng'] = Convert_to_lower_case(row['eng'])
        row['fra'] = Convert_to_lower_case(row['fra'])

In [59]:
Convert_to_lower_cases_for(lines)

In [60]:
lines.head()

,eng,fra
0,i forgot to turn off the tv before going to bed .,j'ai oublié d'éteindre la télé avant d'aller m...
1,tom is approximately the same weight as mary .,tom fait environ le même poids que mary .
2,"if you can't stand the heat , get out of the ...","si vous ne supportez pas la chaleur , sortez ..."
3,has anyone in your family ever been arrested ?,est-ce que quelqu'un dans votre famille a déjà...
4,i have to finish up some things before i go .,je dois finir deux trois trucs avant d'y aller .


### 디코더에 시작토큰과 종료 토큰 삽입

In [61]:
sos_token = '<sos>'
eos_token = '<eos>'
lines['fra'] = lines.fra.apply(lambda x : '<sos> '+ x + ' <eos>')
print('전체 샘플의 수 :',len(lines))
lines.sample(5)

전체 샘플의 수 : 33000


,eng,fra
25842,do you think a little salt would improve the f...,<sos> crois-tu qu'un peu de sel améliorera la ...
29821,we finally have you where we want you .,<sos> nous vous avons finalement mené là où no...
27230,they said he was still weak from a recent sick...,"<sos> ils l'ont dit encore faible , suite à u..."
10791,you are the last person i would have expected ...,<sos> tu es la dernière personne que je m'atte...
7691,many architectural monstrosities are seen in t...,<sos> de nombreuses monstruosités architectura...


3. 띄어쓰기 단위로 토큰화를 수행하세요.

띄어쓰기 단위로 토큰화를 수행해서 단어를 분리하는 작업을 해주세요. 기계는 이렇게 분리된 토큰들을 각각 하나의 단어로 인식할 수 있게 됩니다.

> 토큰화 전 : 'he is a good boy !'  
> 토큰화 후 : ['he', 'is', 'a', 'good', 'boy', '!']

In [62]:
def Tokenize_by_word(sentence: str) -> list:
    return sentence.split()

In [63]:
assert Tokenize_by_word('he is a good boy !') == ['he', 'is', 'a', 'good', 'boy', '!'], 'word tokenizer error.'

In [64]:
def Tokenize_by_word_for(df: pd.DataFrame) -> (list, list): # english, french
    encode_sentences = []
    decode_sentences = []
    for index, row in df.iterrows():
        encode_tokens = Tokenize_by_word(row['eng'])
        decode_tokens = Tokenize_by_word(row['fra'])
        encode_sentences.append(encode_tokens)
        decode_sentences.append(decode_tokens)
    return encode_sentences, decode_sentences

In [65]:
encode, decode = Tokenize_by_word_for(lines)

### Step 2. 디코더의 문장에 시작 토큰과 종료 토큰을 넣어주세요.
---

글자 단위 번역기를 구현할 때와 마찬가지로 디코더의 입력 시퀀스 맨 앞에는 시작을 의미하는 토큰인 <sos>가 필요합니다. 그리고 교사 강요를 수행할 때, 디코더의 실제값이 되는 디코더의 레이블 시퀀스에는 종료를 의미하는 종료 토큰 <eos>가 필요합니다.
예를 들어 번역 문장이 "Courez!" 였다고 한다면, Step 1을 거친 후에는 다음과 같은 결과를 얻습니다.

> Step 1을 수행한 후 : ['courez', '!']

In [66]:
encode

[['i',
  'forgot',
  'to',
  'turn',
  'off',
  'the',
  'tv',
  'before',
  'going',
  'to',
  'bed',
  '.'],
 ['tom', 'is', 'approximately', 'the', 'same', 'weight', 'as', 'mary', '.'],
 ['if',
  'you',
  "can't",
  'stand',
  'the',
  'heat',
  ',',
  'get',
  'out',
  'of',
  'the',
  'kitchen',
  '.'],
 ['has', 'anyone', 'in', 'your', 'family', 'ever', 'been', 'arrested', '?'],
 ['i',
  'have',
  'to',
  'finish',
  'up',
  'some',
  'things',
  'before',
  'i',
  'go',
  '.'],
 ['can', 'you', 'explain', 'the', 'rules', 'to', 'me', ',', 'please', '?'],
 ['i',
  "don't",
  'have',
  'time',
  'to',
  'say',
  'this',
  'twice',
  ',',
  'so',
  'listen',
  'carefully',
  '.'],
 ['please', "don't", 'blow', 'your', 'nose', 'on', 'the', 'tablecloth', '.'],
 ["i'd",
  'like',
  'to',
  'go',
  'through',
  'just',
  'one',
  'day',
  'without',
  'being',
  'told',
  'i',
  'look',
  'like',
  'my',
  'brother',
  '.'],
 ['lake', 'biwa', 'is', 'the', 'largest', 'lake', 'in', 'japan', '

이 문장에 대해서 각각 디코더의 입력 시퀀스와 레이블 시퀀스를 만들면 다음과 같습니다.

> 입력 시퀀스 : ['< sos >', 'courez', '!']  
> 레이블 시퀀스 : ['courez', '!', '< eos >']

In [67]:
decode_input = [sentence[:-1] for sentence in decode]
decode_target = [sentence[1:] for sentence in decode]

In [68]:
decode_input[0:5]

[['<sos>',
  "j'ai",
  'oublié',
  "d'éteindre",
  'la',
  'télé',
  'avant',
  "d'aller",
  'me',
  'coucher',
  '.'],
 ['<sos>',
  'tom',
  'fait',
  'environ',
  'le',
  'même',
  'poids',
  'que',
  'mary',
  '.'],
 ['<sos>',
  'si',
  'vous',
  'ne',
  'supportez',
  'pas',
  'la',
  'chaleur',
  ',',
  'sortez',
  'de',
  'la',
  'cuisine',
  '.'],
 ['<sos>',
  'est-ce',
  'que',
  "quelqu'un",
  'dans',
  'votre',
  'famille',
  'a',
  'déjà',
  'été',
  'arrêté',
  '?'],
 ['<sos>',
  'je',
  'dois',
  'finir',
  'deux',
  'trois',
  'trucs',
  'avant',
  "d'y",
  'aller',
  '.']]

In [69]:
decode_target[0:5]

[["j'ai",
  'oublié',
  "d'éteindre",
  'la',
  'télé',
  'avant',
  "d'aller",
  'me',
  'coucher',
  '.',
  '<eos>'],
 ['tom',
  'fait',
  'environ',
  'le',
  'même',
  'poids',
  'que',
  'mary',
  '.',
  '<eos>'],
 ['si',
  'vous',
  'ne',
  'supportez',
  'pas',
  'la',
  'chaleur',
  ',',
  'sortez',
  'de',
  'la',
  'cuisine',
  '.',
  '<eos>'],
 ['est-ce',
  'que',
  "quelqu'un",
  'dans',
  'votre',
  'famille',
  'a',
  'déjà',
  'été',
  'arrêté',
  '?',
  '<eos>'],
 ['je',
  'dois',
  'finir',
  'deux',
  'trois',
  'trucs',
  'avant',
  "d'y",
  'aller',
  '.',
  '<eos>']]

참고로 Step 2가 반드시 Step 1이 끝난 후에 이루어질 필요는 없습니다!  
Step 1을 수행하는 중간에 수행해도 상관없습니다.

### Step 3. 케라스의 토크나이저로 텍스트를 숫자로 바꿔보세요.
---

딥러닝 모델은 텍스트가 아닌 숫자를 처리합니다. 케라스 토크나이저를 사용해서 각 단어를 고유한 정수로 바꿔보세요.
케라스 토크나이저의 사용법은 아래의 링크에서 2. 케라스(Keras)의 텍스트 전처리에 설명되어 있습니다.

[위키독스](https://wikidocs.net/31766)  

위 링크의 가이드를 통해서 영어와 프랑스어에 대한 토크나이저를 각각 생성하고, tokenizer.texts_to_sequences()를 사용하여 모든 샘플에 대해서 정수 시퀀스로 변환해보세요.

In [70]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer

In [71]:
def tokenize(corpus):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(corpus)
    tensor = tokenizer.texts_to_sequences(corpus)        
    return tensor, tokenizer

In [72]:
encoder_input, encode_tokenizer = tokenize(encode)
decoder_all , decode_tokenizer = tokenize(decode)

decoder_input = decode_tokenizer.texts_to_sequences(decode_input)
decoder_input = tf.keras.preprocessing.sequence.pad_sequences(decoder_input, padding='post')

decoder_target = decode_tokenizer.texts_to_sequences(decode_target)
decoder_target = tf.keras.preprocessing.sequence.pad_sequences(decoder_target, padding='post')

### 원-핫 인코딩으로 정수값을 벡터화

In [75]:
print('영어 단어수:', len(encode_tokenizer.word_index))
print('프랑스어 단어수:', len(decode_tokenizer.word_index))

영어 단어수: 12004
프랑스어 단어수: 18747


In [76]:
encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)
print('영어 데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 출력데이터의 크기(shape) :',np.shape(decoder_target))

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (33000,) + inhomogeneous part.

In [41]:
encoded_seq_len = max([len(sentence) for sentence in encoder_input])
decoded_seq_len = max([len(sentence) for sentence in decoder_all])
print('영어 문장 최대 길이', encoded_seq_len)
print('프랑스 문장 최대 길이', decoded_seq_len)

영어 문장 최대 길이 51
프랑스 문장 최대 길이 61


In [42]:
print(encoder_input[0].shape) #  (샘플의 수 × 샘플의 길이 × 단어장의 크기) 

(51, 12005)


In [43]:
print(decoder_input[:3])
print(decoder_target[:3])

[[[0. 1. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]

 [[0. 1. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]

 [[0. 1. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]]
[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]]


### 훈련세트 - 검증 세트 분리

In [ ]:
encoder_input = pad_sequences(encoder_input, maxlen = max_eng_seq_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen = max_fra_seq_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen = max_fra_seq_len, padding='post')

In [34]:
n_of_val = 3000

encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

print('영어 학습데이터의 크기(shape) :',np.shape(encoder_input_train))
print('프랑스어 학습 입력데이터의 크기(shape) :',np.shape(decoder_input_train))
print('프랑스어 학습 출력데이터의 크기(shape) :',np.shape(decoder_target_train))

영어 학습데이터의 크기(shape) : (30000, 51, 12006)
프랑스어 학습 입력데이터의 크기(shape) : (30000, 60, 18749)
프랑스어 학습 출력데이터의 크기(shape) : (30000, 60, 18749)


### Step 4. 임베딩 층(Embedding layer) 사용하기

이번에는 입력이 되는 각 단어를 임베딩 층을 사용하여 벡터화하겠습니다.
임베딩 층을 사용하는 방법과 그 설명에 대해서는 아래의 링크의 1. 케라스 임베딩 층(Keras Embedding layer) 을 참고하세요.

[위키독스](https://wikidocs.net/33793)

실제 번역기 구현을 위해서 사용할 수 있는 인코더 코드의 예시는 다음과 같습니다. 이를 통해서 인코더와 디코더의 임베딩 층을 각각 구현해보세요.

In [35]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model

#### 인코더

In [36]:
# 입력 텐서 정의 및 생성.
eng_vocab_size = 12006
fra_vocab_size = 18749

encoder_inputs = Input(shape=(None,))

In [37]:
enc_emb =  Embedding(eng_vocab_size, 9)(encoder_inputs)

Metal device set to: Apple M2


2022-12-15 23:04:37.359745: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-12-15 23:04:37.359820: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [38]:
# hidden size가 256인 인코더의 LSTM 셀 생성
encoder_lstm = LSTM(units = 256, return_state = True)

In [39]:
# 디코더로 전달할 hidden state, cell state를 리턴. encoder_outputs은 여기서는 불필요.
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)

In [40]:
# hidden state와 cell state를 다음 time step으로 전달하기 위해서 별도 저장.
encoder_states = [state_h, state_c]

#### 디코더

In [41]:
# 디코더의 입력 텐서 생성.
decoder_inputs = Input(shape=(None, fra_vocab_size))

In [42]:
# hidden size가 256인 디코더의 LSTM 셀 생성
decoder_lstm = LSTM(units = 256, return_sequences = True, return_state=True)

In [43]:
# decoder_outputs는 모든 time step의 hidden state : return_sequence = True
decoder_outputs, _, _= decoder_lstm(decoder_inputs, initial_state = encoder_states)

#### 출력층

In [44]:
decoder_softmax_layer = Dense(fra_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

### Step 5. 모델 구현하기
---

글자 단위 번역기에서 구현한 모델을 참고로 단어 단위 번역기의 모델을 완성시켜보세요! 이때는 label이 integer 값이므로 categorical entropy loss가 아닌 sparse categorical entropy loss를 사용합니다.

In [45]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [46]:
model.compile(optimizer="rmsprop", loss="categorical_crossentropy")

In [47]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, None, 9)      108054      ['input_1[0][0]']                
                                                                                                  
 input_2 (InputLayer)           [(None, None, 18749  0           []                               
                                )]                                                                
                                                                                                  
 lstm (LSTM)                    [(None, 256),        272384      ['embedding[0][0]']          

### 모델 훈련하기

In [50]:
model.fit(x=[encoder_input_train, decoder_input_train], y=decoder_target_train, batch_size=1, epochs=1)

TypeError: fit() got an unexpected keyword argument 'device'

### Step 6. 모델 평가하기
---
단어 단위 번역기를 이용하여 훈련 데이터의 샘플과 테스트 데이터의 샘플로 번역 문장을 만들어보고 정답 문장과 번역 문장을 비교해보세요. 이전 스텝들에서 우리가 공부했던 모델의 경우 글자 단위에서 구현된 번역기이며 현재 프로젝트를 진행할 때 사용하는 모델은 단어 단위에서 구현되는 번역기입니다.

> Embedding layer가 추가되기 때문에 학습했던 내용 그대로 사용할 경우 shape에서 error가 발생합니다.
> decode sentence를 구성할 때 고민해보세요!!

고민하다 풀리지 않을 경우에는 하단 내용 참고해주세요.

In [ ]:
# 문장 디코딩 예시
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)

    # 에 해당하는 원-핫 벡터 생성
    target_seq = np.zeros((1,1)) 
    target_seq[0, 0] = fra2idx['\t']
    
    stop_condition = False
    decoded_sentence = ""

    # stop_condition이 True가 될 때까지 루프 반복
    while not stop_condition:
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        

루브릭  
아래의 기준을 바탕으로 프로젝트를 평가합니다.  
평가문항	상세기준

1. 번역기 모델 학습에 필요한 텍스트 데이터 전처리가 잘 이루어졌다.  
구두점, 대소문자, 띄어쓰기 등 번역기 모델에 요구되는 전처리가 정상적으로 진행되었다.  

2. seq2seq 기반의 번역기 모델이 정상적으로 구동된다.  
seq2seq 모델 훈련결과를 그래프로 출력해보고, validation loss그래프가 우하향하는 경향성을 보이며 학습이 진행됨이 확인되었다.

3. 테스트 결과 의미가 통하는 수준의 번역문이 생성되었다.  
테스트용 디코더 모델이 정상적으로 만들어졌으며, input(영어)와 output(프랑스어) 모두 한글로 번역해서 결과를 출력해보았고, 둘의 내용이 유사함을 확인하였다.
